In [108]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import my_api_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [211]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Category 2": [],
    "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for i, city, state in zip(len(offset), random_rural_city["NAME"], random_rural_city["STNAME"]):
    params = {
        'limit': 50,
        'offset': offset[i],
        'location': f"{city}, {state}",
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()

    for business in range(len(response["businesses"])):
        try:
            rural_dict["Searched"].append(f"{city}, {state}")
            rural_dict["City"].append(response["businesses"][business]["location"]["city"])
            rural_dict["State"].append(response["businesses"][business]["location"]["state"])
            rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            rural_dict["Restaurant"].append(response["businesses"][business]["name"])
#             rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
            rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
            rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
            rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            rural_dict["Searched"].append("No City Found")
            rural_dict["City"].append("No City Found")
            rural_dict["State"].append("No State Found")
            rural_dict["Lat"].append("No Lat Found")
            rural_dict["Lng"].append("No Lng Found")
            rural_dict["Restaurant"].append("No Restaurant Found")
            rural_dict["Category"].append("No Categories")
            rural_dict["Transactions"].append("No Transactions Found")
        except IndexError:
            rural_dict["Category 2"].append("")
            rural_dict["Category 3"].append("")

TypeError: 'int' object is not iterable

In [212]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head(100)

,Searched,City,State,Lat,Lng,Restaurant,Category,Category 2,Category 3,Transactions


In [166]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Category 2": [],
    "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city, state in zip(random_urban_city["NAME"], random_urban_city["STNAME"]):
    params = {
        'limit': 50,
        'offset': 50,
        'location': f"{city}, {state}",
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            urban_dict["Searched"].append(f"{city}, {state}")
            urban_dict["City"].append(response["businesses"][business]["location"]["city"])
            urban_dict["State"].append(response["businesses"][business]["location"]["state"])
            urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            urban_dict["Restaurant"].append(response["businesses"][business]["name"])
#             urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            urban_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
            urban_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
            urban_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
            urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            urban_dict["Searched"].append("No City Found")
            urban_dict["City"].append("No City Found")
            urban_dict["State"].append("No State Found")
            urban_dict["Lat"].append("No Lat Found")
            urban_dict["Lng"].append("No Lng Found")
            urban_dict["Restaurant"].append("No Restaurant Found")
            urban_dict["Category"].append("No Categories")
            urban_dict["Transactions"].append("No Transactions Found")
        except IndexError:
            urban_dict["Category 2"].append("")
            urban_dict["Category 3"].append("")

In [167]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()
urban.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Category 2,Category 3,Transactions
0,"Madison city, Alabama",Madison,AL,34.7011,-86.7418,Jack's,Burgers,,,[]
1,"Madison city, Alabama",Madison,AL,34.7524,-86.7464,Ricks Barbecue - Madison,Barbeque,Southern,,[]
2,"Madison city, Alabama",Madison,AL,34.7061,-86.7439,Old Mexico Cantina,Mexican,,,[]
3,"Madison city, Alabama",Madison,AL,34.7526,-86.7515,ChuckWagon BBQ,Barbeque,,,[]
4,"Madison city, Alabama",Madison,AL,34.7073,-86.7506,Culver's,Burgers,,Fast Food,[]


In [206]:
urban.to_csv("../urban.csv")
rural.to_csv("../rural.csv")

In [177]:
urban_vegan = urban[(urban["Category"] == "Vegan")|(urban["Category 2"] == "Vegan")|(urban["Category 3"] == "Vegan")]
urban_vegan.head(100)


,Searched,City,State,Lat,Lng,Restaurant,Category,Category 2,Category 3,Transactions
188,"Bibb County, Georgia",Macon,GA,32.8368,-83.6712,Harmony Vibes,Vegan,,Burgers,[]
194,"Oceanside city, California",Oceanside,CA,33.1831,-117.332,Everbowl,Acai Bowls,Fast Food,Vegan,[]
593,"Marion County, Indiana",Indianapolis,IN,39.7812,-86.1096,10th Street Diner,Vegan,,,[]
668,"Toms River township, New Jersey",Toms River,NJ,39.9973,-74.2112,Cafe Diem,Food Trucks,,Vegan,[]
679,"Toms River township, New Jersey",Toms River,NJ,39.9638,-74.189,Checkers,Burgers,Vegan,,[]
831,"Hudson County, New Jersey",Jersey City,NJ,40.7212,-74.0459,Pet Shop,Bars,,Vegan,[delivery]
1093,"Westminster city, Colorado",Denver,CO,39.7568,-104.95,Vegan Van,Vegan,American (Traditional),,[delivery]
1213,"Ramapo town, New York",Kerhonkson,NY,41.7866,-74.2763,BABA Food Truck,Vegan,Beer Gardens,,[delivery]
1408,"Margate city, Florida",Ft Lauderdale,FL,26.1243,-80.1289,Reggae Beets Gourmet Food Truck,Juice Bars & Smoothies,Shaved Ice,Vegan,[]
1736,"Salem city, Oregon",Salem,OR,44.9386,-123.039,Infinity Room,Music Venues,,Vegan,None
